# Adding Latitude and Longitude Coordinates

The objective is to find the geographic location of each headline in latitude and longitude coordinates from the city/country names.

# Workflow

1. Load in the pandas DataFrame with headline, countries, and cities.
   * If a headline contains multiple cities/countries, decide which single one to keep.
2. For each city/country, match the name to the latitude and longitude in geonamescache.
   * You can use the function gc.get_cities_by_names_ _(“city_name”).
   * Some cities will return multiple matches with the previous function in different countries. You’ll have to decide which city to keep based on a heuristic (rule of thumb).
   * If you have trouble, work with a single problematic city until you figure it out, then write a function to apply on all headlines.
3. Add longitude and latitude coordinates to your DataFrame for each headline.
   * It will be helpful to get the countrycode of each headline at this point.
   * If you were not able to find many countries, think about dropping the column. You also need to decide what to do with headlines that have no coordinates.
   * You should end up with over 600 headlines that have geographic coordinates.

The prerequisite to this part is the output `exercise1-output.csv' from exercise 1.

In [1]:
import pandas as pd

df = pd.read_csv('exercise1-output.csv')         # requires the output from exercise 1
df

,headline,country,city
0,Zika Outbreak Hits Miami,NaN,Miami
1,Could Zika Reach New York City?,NaN,New York City
2,First Case of Zika in Miami Beach,NaN,Miami Beach
3,"Mystery Virus Spreads in Recife, Brazil",Brazil,Recife
4,Dallas man comes down with case of Zika,NaN,Dallas
...,...,...,...
645,Rumors about Rabies spreading in Jerusalem hav...,NaN,Jerusalem
646,More Zika patients reported in Indang,NaN,Indang
647,Suva authorities confirmed the spread of Rotav...,NaN,Suva
648,More Zika patients reported in Bella Vista,NaN,Bella Vista


Let's check whether there are multiple countries/cities listed in the headlines.

In [2]:
import geonamescache
import re
from text_unidecode import unidecode

gnc = geonamescache.GeonamesCache()
countries = [unidecode(v['name']) for v in gnc.get_countries().values()]
countries = sorted(countries, key=len, reverse=True)
country_regex = re.compile(f"\\b({'|'.join(countries)})\\b")

df.loc[:, 'headline2'] = df.headline.str.replace(country_regex, '', regex=True, n=1)
df.loc[:, 'country2'] = df.headline2.str.extract(country_regex, expand=False)
df.loc[pd.notna(df.country2),]

,headline,country,city,headline2,country2


Repeat the same for city:

In [3]:
cities = [unidecode(v['name']) for v in gnc.get_cities().values()]
cities = sorted(cities, key=len, reverse=True)
city_regex = re.compile(f"\\b({'|'.join(cities)})\\b")

df.loc[:, 'headline2'] = df.headline.str.replace(city_regex, '', regex=True, n=1)
df.loc[:, 'city2'] = df.headline2.str.extract(city_regex, expand=False)
df.loc[pd.notna(df.city2),]

,headline,country,city,headline2,country2,city2


As all headlines do not mention multiple countries or cities, drop xxx2 columns:

In [4]:
df.drop(['headline2', 'city2', 'country2'], axis=1, inplace=True)
df

,headline,country,city
0,Zika Outbreak Hits Miami,NaN,Miami
1,Could Zika Reach New York City?,NaN,New York City
2,First Case of Zika in Miami Beach,NaN,Miami Beach
3,"Mystery Virus Spreads in Recife, Brazil",Brazil,Recife
4,Dallas man comes down with case of Zika,NaN,Dallas
...,...,...,...
645,Rumors about Rabies spreading in Jerusalem hav...,NaN,Jerusalem
646,More Zika patients reported in Indang,NaN,Indang
647,Suva authorities confirmed the spread of Rotav...,NaN,Suva
648,More Zika patients reported in Bella Vista,NaN,Bella Vista


In [5]:
from collections import Counter

city_count = {k: v for k, v in Counter(cities).items()}
sorted(list(city_count.items()), key=lambda x: x[1], reverse=True)[:10]

[('Springfield', 8),
 ('San Fernando', 7),
 ('San Pedro', 7),
 ('Richmond', 7),
 ('San Francisco', 6),
 ('Santa Cruz', 6),
 ('Burlington', 6),
 ('San Carlos', 6),
 ('San Marcos', 6),
 ('Mercedes', 6)]

Because the same city name can be found in different countries, we examine the cities GeonamesCache has to confirm that. Such duplication implicates the tagging of geographic coordinate to the headlines: other than the city name, the country matters as well.

In [6]:
[v['countrycode'] for v in gnc.get_cities().values() if v['name'] == 'Mercedes']

['AR', 'AR', 'CR', 'PH', 'US', 'UY']

As the output above shows, Mercedes covers quite a few countries.  Let's check whether the same happens to countries:

In [7]:
country_count = {k: v for k, v in Counter(countries).items()}
sorted(list(country_count.items()), key=lambda x: x[1], reverse=True)[:10]

[('South Georgia and the South Sandwich Islands', 1),
 ('United States Minor Outlying Islands', 1),
 ('Bonaire, Saint Eustatius and Saba ', 1),
 ('Heard Island and McDonald Islands', 1),
 ('Democratic Republic of the Congo', 1),
 ('Saint Vincent and the Grenadines', 1),
 ('British Indian Ocean Territory', 1),
 ('French Southern Territories', 1),
 ('Saint Pierre and Miquelon', 1),
 ('Central African Republic', 1)]

Luckily, countries do not have the same problem. With these in mind, let's proceed with creating a data frame with country, city, and coordinates:

In [8]:
country_df = pd.DataFrame([(k, unidecode(v['name'])) for k, v in gnc.get_countries().items()],
                          columns=['countrycode', 'country'])
city_df = pd.DataFrame([(unidecode(v['countrycode']), unidecode(v['name']), v['longitude'], v['latitude'], v['population'])
                        for v in gnc.get_cities().values()],
                       columns=['countrycode', 'city', 'longitude', 'latitude', 'population'])
country_city = country_df.merge(city_df, on='countrycode')
country_city

,countrycode,country,city,longitude,latitude,population
0,AD,Andorra,Andorra la Vella,1.52109,42.50779,20430
1,AE,United Arab Emirates,Umm Al Quwain City,55.55517,25.56473,62747
2,AE,United Arab Emirates,Ras Al Khaimah City,55.94320,25.78953,351943
3,AE,United Arab Emirates,Zayed City,53.70522,23.65416,63482
4,AE,United Arab Emirates,Khawr Fakkan,56.34199,25.33132,40677
...,...,...,...,...,...,...
24331,ZW,Zimbabwe,Bulawayo,28.58333,-20.15000,699385
24332,ZW,Zimbabwe,Bindura,31.33056,-17.30192,37423
24333,ZW,Zimbabwe,Beitbridge,30.00000,-22.21667,26459
24334,ZW,Zimbabwe,Epworth,31.14750,-17.89000,123250


Remember that some cities are repeated multiple times? For example, Mercedes is repeated 6 times and 2 of that are found within Argentina itself:

In [9]:
country_city.loc[country_city.city == 'Mercedes']

,countrycode,country,city,longitude,latitude,population
177,AR,Argentina,Mercedes,-59.43068,-34.65145,52949
178,AR,Argentina,Mercedes,-58.07519,-29.18416,30649
4220,CR,Costa Rica,Mercedes,-84.13396,10.00695,26007
16273,PH,Philippines,Mercedes,123.01090,14.10930,16991
21536,US,United States,Mercedes,-97.91361,26.14980,16657
23774,UY,Uruguay,Mercedes,-58.03047,-33.25240,42359


There's no way to disambiguate one Mercedes from another within Argentina purely from the headlines, using the largest population probably might be a heuristic that we could use: the more populous the city, the more likely non-locals will visit or transit through that city, thus increasing the chance of bringing in/out virus(es). Let's trim `country_city` to keep only the most populous city:

In [10]:
country_city.sort_values(by=['country', 'city', 'population'], ascending=False, inplace=True)
country_city.loc[country_city.city == 'Mercedes']

,countrycode,country,city,longitude,latitude,population
23774,UY,Uruguay,Mercedes,-58.03047,-33.25240,42359
21536,US,United States,Mercedes,-97.91361,26.14980,16657
16273,PH,Philippines,Mercedes,123.01090,14.10930,16991
4220,CR,Costa Rica,Mercedes,-84.13396,10.00695,26007
177,AR,Argentina,Mercedes,-59.43068,-34.65145,52949
178,AR,Argentina,Mercedes,-58.07519,-29.18416,30649


The sorting looks correctly, let's proceed with taking the first in each group:

In [11]:
cc = country_city.groupby(['countrycode', 'city']).first().reset_index()
cc.loc[cc.city == 'Mercedes']

,countrycode,city,country,longitude,latitude,population
243,AR,Mercedes,Argentina,-59.43068,-34.65145,52949
4148,CR,Mercedes,Costa Rica,-84.13396,10.00695,26007
16123,PH,Mercedes,Philippines,123.01090,14.10930,16991
21756,US,Mercedes,United States,-97.91361,26.14980,16657
23068,UY,Mercedes,Uruguay,-58.03047,-33.25240,42359


Now, merge this with headlines:

In [12]:
df.merge(cc, on=['country', 'city'])

,headline,country,city,countrycode,longitude,latitude,population
0,"Mystery Virus Spreads in Recife, Brazil",Brazil,Recife,BR,-34.88111,-8.05389,1478098
1,Zika cases in Vietnam's Ho Chi Minh City surge,Vietnam,Ho Chi Minh City,VN,106.62965,10.82302,3467331
2,Thailand-Zika Virus in Bangkok,Thailand,Bangkok,TH,100.50144,13.75398,5104476
3,"Zika outbreak in Piracicaba, Brazil",Brazil,Piracicaba,BR,-47.64917,-22.72528,342209
4,"Zika surfaces in Klang, Malaysia",Malaysia,Klang,MY,101.44333,3.03667,879867
5,Rumors about Meningitis spreading in Guatemala...,Guatemala,Guatemala City,GT,-90.51327,14.64072,994938
6,Belize City under threat from Zika,Belize,Belize City,BZ,-88.19756,17.49952,61461
7,"Student sick in Campinas, Brazil",Brazil,Campinas,BR,-47.06083,-22.90556,1031554
8,Zika outbreak spreads to Mexico City,Mexico,Mexico City,MX,-99.12766,19.42847,12294193
9,"New Zika Case in Kota Kinabalu, Malaysia",Malaysia,Kota Kinabalu,MY,116.07240,5.97490,457326


As only a few headlines states both the country and city, we need to do better than naively merging the two data frames. We will use the same heuristics when merging just on one field: use the more populous country or city:

In [13]:
both = df.merge(cc, on=['city', 'country'], sort=False)
country_only = df.loc[pd.notna(df.country) & pd.isna(df.city)].merge(cc.drop('city', axis=1), on='country', sort=False)

populous_city = cc.drop('country', axis=1).sort_values(by=['population', 'city'], ascending=False).groupby('city').first()
city_only = df.loc[pd.isna(df.country) & pd.notna(df.city)].merge(populous_city, on='city', sort=False)

result = pd.concat([both, country_only, city_only])
result

,headline,country,city,countrycode,longitude,latitude,population
0,"Mystery Virus Spreads in Recife, Brazil",Brazil,Recife,BR,-34.88111,-8.05389,1478098
1,Zika cases in Vietnam's Ho Chi Minh City surge,Vietnam,Ho Chi Minh City,VN,106.62965,10.82302,3467331
2,Thailand-Zika Virus in Bangkok,Thailand,Bangkok,TH,100.50144,13.75398,5104476
3,"Zika outbreak in Piracicaba, Brazil",Brazil,Piracicaba,BR,-47.64917,-22.72528,342209
4,"Zika surfaces in Klang, Malaysia",Malaysia,Klang,MY,101.44333,3.03667,879867
...,...,...,...,...,...,...,...
588,Rumors about Rabies spreading in Jerusalem hav...,NaN,Jerusalem,IL,35.21633,31.76904,801000
589,More Zika patients reported in Indang,NaN,Indang,PH,120.87694,14.19528,41159
590,Suva authorities confirmed the spread of Rotav...,NaN,Suva,FJ,178.44149,-18.14161,77366
591,More Zika patients reported in Bella Vista,NaN,Bella Vista,DO,-69.94540,18.45539,175683


Save the output as exercise2-output.csv:

In [14]:
with open('exercise2-output.csv', 'w') as fout:
    fout.write(result.to_csv(index=False))